In [1]:
import pandas as pd

studentsfile = "Resources/students_complete.csv"
schoolsfile = "Resources/schools_complete.csv"

students_df = pd.read_csv(studentsfile)
schools_df = pd.read_csv(schoolsfile)

In [2]:
students_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [3]:
schools_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [4]:
#find totals by calling count on any column in each data frame
totalschools = schools_df['school_name'].count()
totalstudents = students_df['Student ID'].count()

#basic math for sum and averages
totalbudget = schools_df['budget'].sum()
read_mean = students_df['reading_score'].mean()
math_mean = students_df['math_score'].mean()

#create new df of just passing students, count how many (in any column) and divide by total
read_pass_df = students_df.loc[students_df['reading_score'] >= 70]
read_pass = (read_pass_df['reading_score'].count())/ (students_df['reading_score'].count())

#repeat process for math passing
math_pass_df = students_df.loc[students_df['math_score'] >= 70]
math_pass = (math_pass_df['math_score'].count())/ (students_df['reading_score'].count())

#check if those who passed reading also passed math
both_pass_df = math_pass_df.loc[math_pass_df['reading_score'] >= 70]
both_pass = (both_pass_df['reading_score'].count()/ students_df['reading_score'].count())
both_pass

0.6517232575950983

In [5]:
#create distric summary df
summary_df = pd.DataFrame({"Total Schools": [totalschools],
              "Total Students": [totalstudents],
              "Total Budget": [totalbudget],
              "Passing % for Reading": [read_pass],
              "Passing % for Math": [math_pass],
              "Passing % for Both": [both_pass]})
summary_df.reset_index(drop=True)
summary_df


,Total Schools,Total Students,Total Budget,Passing % for Reading,Passing % for Math,Passing % for Both
0,15,39170,24649428,0.858055,0.749809,0.651723


In [6]:
#moving on to school summaries
#calculates per student budget and creates a new column
schools_df['Per Student Budget'] = schools_df['budget']/schools_df['size']
#schools_df['% Passing for Reading'] = 
schools_df.head()

,School ID,school_name,type,size,budget,Per Student Budget
0,0,Huang High School,District,2917,1910635,655.0
1,1,Figueroa High School,District,2949,1884411,639.0
2,2,Shelton High School,Charter,1761,1056600,600.0
3,3,Hernandez High School,District,4635,3022020,652.0
4,4,Griffin High School,Charter,1468,917500,625.0


In [7]:
grouped_students = students_df.groupby('school_name')
read_by_school = pd.DataFrame(grouped_students['reading_score'].mean())
math_by_school = pd.DataFrame(grouped_students['math_score'].mean())

In [8]:

#merge new dfs with school summary to add new calculations
merge1_df = pd.merge(schools_df, read_by_school, on="school_name")
merged_df = pd.merge(merge1_df, math_by_school, on="school_name")
merged_df = merged_df.rename(columns={'reading_score': 'Average Reading Score', 'math_score':'Average Math Score'})

In [9]:
#create a new table that only shows passing students (already have)
#group that by school
#compare passing vs school size (already in df)
grouped_read_pass = read_pass_df.groupby(read_pass_df['school_name']).count()

del grouped_read_pass['Student ID']
del grouped_read_pass['student_name']
del grouped_read_pass['grade']
del grouped_read_pass['math_score']
del grouped_read_pass['gender']

grouped_read_pass.head()

grouped_math_pass = math_pass_df.groupby(math_pass_df['school_name']).count()


del grouped_math_pass['Student ID']
del grouped_math_pass['student_name']
del grouped_math_pass['grade']
del grouped_math_pass['reading_score']
del grouped_math_pass['gender']

grouped_math_pass.head()

#merge these two together
firstgroup_passing = pd.merge(grouped_read_pass, grouped_math_pass, on="school_name")
firstgroup_passing.head()

,reading_score,math_score
school_name,,
Bailey High School,4077,3318
Cabrera High School,1803,1749
Figueroa High School,2381,1946
Ford High School,2172,1871
Griffin High School,1426,1371


In [10]:
both_pass_group = both_pass_df.groupby(both_pass_df['school_name']).count()

del both_pass_group['Student ID']
del both_pass_group['student_name']
del both_pass_group['grade']
del both_pass_group['math_score']
del both_pass_group['reading_score']

both_pass_group.head()

group_passing = pd.merge(firstgroup_passing, both_pass_group, on='school_name')
group_passing


,reading_score,math_score,gender
school_name,,,
Bailey High School,4077,3318,2719
Cabrera High School,1803,1749,1697
Figueroa High School,2381,1946,1569
Ford High School,2172,1871,1487
Griffin High School,1426,1371,1330
Hernandez High School,3748,3094,2481
Holden High School,411,395,381
Huang High School,2372,1916,1561
Johnson High School,3867,3145,2549


In [11]:
#merge both sets with the schools summary
school_summary_df = pd.merge(merged_df, group_passing, on="school_name")
school_summary_df

#calculate passing percentages with size!
school_summary_df['Math Passing %'] = school_summary_df['math_score']/school_summary_df['size']
school_summary_df['Reading Passing %'] = school_summary_df['reading_score']/school_summary_df['size']
school_summary_df['Total Passing %'] = school_summary_df['gender']/school_summary_df['size']


#remove 
del school_summary_df['reading_score']
del school_summary_df['math_score']
del school_summary_df['gender']
school_summary_df

,School ID,school_name,type,size,budget,Per Student Budget,Average Reading Score,Average Math Score,Math Passing %,Reading Passing %,Total Passing %
0,0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,0.656839,0.813164,0.535139
1,1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,0.659885,0.807392,0.532045
2,2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,0.938671,0.958546,0.898921
3,3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,0.667530,0.808630,0.535275
4,4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,0.933924,0.971390,0.905995
5,5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,0.938677,0.965396,0.905826
6,6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,0.941335,0.970398,0.913348
7,7,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,0.666801,0.819333,0.546423
8,8,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,0.925059,0.962529,0.892272
9,9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,0.945946,0.959459,0.905405


In [12]:
#time to move on to the next table, a list of top 5 schools based on total passing %

top5 = school_summary_df.sort_values('Total Passing %', ascending=False).head()
top5

,School ID,school_name,type,size,budget,Per Student Budget,Average Reading Score,Average Math Score,Math Passing %,Reading Passing %,Total Passing %
6,6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,0.941335,0.970398,0.913348
14,14,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,0.932722,0.973089,0.909480
4,4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,0.933924,0.971390,0.905995
5,5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,0.938677,0.965396,0.905826
9,9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,0.945946,0.959459,0.905405


In [13]:
bot5 = school_summary_df.sort_values('Total Passing %').head()
bot5

,School ID,school_name,type,size,budget,Per Student Budget,Average Reading Score,Average Math Score,Math Passing %,Reading Passing %,Total Passing %
11,11,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,0.663666,0.802201,0.529882
1,1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,0.659885,0.807392,0.532045
0,0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,0.656839,0.813164,0.535139
3,3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,0.667530,0.808630,0.535275
12,12,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,0.660576,0.812224,0.535392


In [14]:
#scores based on grade level
#create function to turn 
def schoolscores(df):
    del df['Student ID']
    del df['student_name']
    del df['gender']
    del df['grade']
    
    grouping = df.groupby('school_name')
    returner = pd.DataFrame(grouping.mean())
    return returner

#9th grade
grade_group_9 = students_df.loc[students_df["grade"] == "9th"]
grade_9_cal = schoolscores(grade_group_9)
grade_9_cal

#10th grade
grade_group_10 = students_df.loc[students_df["grade"] == "10th"]
grade_10_cal = schoolscores(grade_group_10)

#11th
grade_group_11 = students_df.loc[students_df["grade"] == "11th"]
grade_11_cal = schoolscores(grade_group_11)

#12th
grade_group_12 = students_df.loc[students_df["grade"] == "12th"]
grade_12_cal = schoolscores(grade_group_12)

In [24]:
grade_9_math = grade_9_cal.iloc[:,1]
grade_10_math = grade_10_cal.iloc[:,1]
grade_11_math = grade_11_cal.iloc[:,1]
grade_12_math = grade_12_cal.iloc[:,1]

mergedMathBySchool = pd.merge(grade_9_math, grade_10_math, on="school_name", suffixes=("_y","_z"))
mergedMathBySchool = pd.merge(mergedMathBySchool, grade_11_math, on="school_name", suffixes=("_a","_b"))
mergedMathBySchool = pd.merge(mergedMathBySchool, grade_12_math, on="school_name", suffixes=("_c","_d"))
mergedMathBySchool.rename(columns={
                                   "math_score_y": "9th Grade",
                                   "math_score_z": "10th Grade",
                                   "math_score_c": "11th Grade",
                                   "math_score_d": "12th Grade",
                                        })



,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [26]:
grade_9_cal

,reading_score,math_score
school_name,,
Bailey High School,81.303155,77.083676
Cabrera High School,83.676136,83.094697
Figueroa High School,81.198598,76.403037
Ford High School,80.632653,77.361345
Griffin High School,83.369193,82.044010
Hernandez High School,80.866860,77.438495
Holden High School,83.677165,83.787402
Huang High School,81.290284,77.027251
Johnson High School,81.260714,77.187857


In [30]:
grade_9_read = grade_9_cal.iloc[:,0]
grade_10_read = grade_10_cal.iloc[:,0]
grade_11_read = grade_11_cal.iloc[:,0]
grade_12_read = grade_12_cal.iloc[:,0]

mergedReadBySchool = pd.merge(grade_9_read, grade_10_read, on="school_name", suffixes=("_y","_z"))
mergedReadBySchool = pd.merge(mergedReadBySchool, grade_11_read, on="school_name", suffixes=("_a","_b"))
mergedReadBySchool = pd.merge(mergedReadBySchool, grade_12_read, on="school_name", suffixes=("_c","_d"))
mergedReadBySchool.rename(columns={
                                   "reading_score_y": "9th Grade",
                                   "reading_score_z": "10th Grade",
                                   "reading_score_c": "11th Grade",
                                   "reading_score_d": "12th Grade",
                                        })


,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [39]:
spending_bins_df = school_summary_df.copy()

bins = [0, 585, 630, 645, 680]
bin_labels = ["<$585", "$585-630", "$630-645","$645-680"]

spending_bins_df["Spending Ranges"] = pd.cut(spending_bins_df["Per Student Budget"], bins, labels=bin_labels)
binso = spending_bins_df.groupby('Spending Ranges')
binso[['Average Reading Score', 'Average Math Score', 'Math Passing %', 'Reading Passing %', 'Total Passing %']].mean()

,Average Reading Score,Average Math Score,Math Passing %,Reading Passing %,Total Passing %
Spending Ranges,,,,,
<$585,83.933814,83.455399,0.934601,0.966109,0.903695
$585-630,83.155286,81.899826,0.871335,0.927182,0.814186
$630-645,81.624473,78.518855,0.734842,0.843918,0.628577
$645-680,81.027843,76.997210,0.661648,0.811340,0.535269


In [38]:
spending_bins_df.head()

,School ID,school_name,type,size,budget,Per Student Budget,Average Reading Score,Average Math Score,Math Passing %,Reading Passing %,Total Passing %,Spending Ranges
0,0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,0.656839,0.813164,0.535139,$645-680
1,1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,0.659885,0.807392,0.532045,$630-645
2,2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,0.938671,0.958546,0.898921,$585-630
3,3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,0.667530,0.808630,0.535275,$645-680
4,4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,0.933924,0.971390,0.905995,$585-630


In [41]:
spending_bins_df1 = school_summary_df.copy()

bins1 = [0, 1000, 2000, 5000]
bin_labels1 = ["Small <1000", "Medium 1000-2000", "Large 2000-5000"]

spending_bins_df1["size"] = pd.cut(spending_bins_df1["Per Student Budget"], bins1, labels=bin_labels1)
binso1 = spending_bins_df.groupby('size')
binso1[['Average Reading Score', 'Average Math Score', 'Math Passing %', 'Reading Passing %', 'Total Passing %']].mean()

,Average Reading Score,Average Math Score,Math Passing %,Reading Passing %,Total Passing %
size,,,,,
427,83.814988,83.803279,0.925059,0.962529,0.892272
962,84.044699,83.839917,0.945946,0.959459,0.905405
1468,83.816757,83.351499,0.933924,0.971390,0.905995
1635,83.848930,83.418349,0.932722,0.973089,0.909480
1761,83.725724,83.359455,0.938671,0.958546,0.898921
1800,83.955000,83.682222,0.933333,0.966111,0.903333
1858,83.975780,83.061895,0.941335,0.970398,0.913348
2283,83.989488,83.274201,0.938677,0.965396,0.905826
2739,80.746258,77.102592,0.683096,0.792990,0.542899
